In [0]:
import zipfile

In [0]:
with zipfile.ZipFile("/content/drive/My Drive/gtsrb-german-traffic-sign.zip","r") as zip_ref:
  zip_ref.extractall("/content/sample_data")

In [0]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from skimage import io
from PIL import Image

In [0]:
from torch.utils.data import Dataset

In [0]:
class CustomDataset(Dataset):

  def __init__(self,csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    y_label = torch.tensor(int(self.annotations.iloc[index,-2]))
    img_path = os.path.join(self.root_dir, self.annotations.iloc[index,-1])
    image = Image.open(img_path)
    

    if self.transform:
      image = self.transform(image)

    return (image, y_label)


In [0]:
from torchvision import transforms

In [0]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [0]:
transform = transforms.Compose([
                  transforms.RandomResizedCrop(size=256),
                  transforms.RandomRotation(degrees=15),
                  transforms.RandomHorizontalFlip(),
                  transforms.ToTensor(),
                  transforms.Normalize(mean, std)
])

In [0]:
trainset = CustomDataset('/content/sample_data/Train.csv','/content/sample_data', transform=transform)

In [0]:
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)

In [0]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape)
print(images[2].shape)
print(labels.shape)

torch.Size([16, 3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([16])


In [0]:
import numpy as np

In [0]:
import matplotlib.pyplot as plt

In [0]:
test_transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
])

In [0]:
testset = CustomDataset('/content/sample_data/Test.csv','/content/sample_data', transform=test_transform)

In [0]:
testloader = DataLoader(testset, batch_size=16, shuffle=True)

In [0]:
device = torch.device('cuda:0')

In [0]:
print(device)

cuda:0


In [0]:
torch.cuda.get_device_name()

'Tesla K80'

In [0]:
from torchvision import models

In [0]:
from torch import optim
import torch.nn as nn

In [0]:
def create_model(n_classes):
  model = models.resnet34(pretrained=True)

  n_features = model.fc.in_features

  model.fc = nn.Sequential(
      nn.Linear(n_features, n_classes),
      nn.Softmax(dim=1)
  )

  return model.to(device)

In [0]:
base_model = create_model(43)

In [0]:
print(base_model)

In [0]:
def evaluation(dataloader):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = base_model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [0]:
opt = optim.SGD(base_model.parameters(),lr=0.001,momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

In [0]:
%%time
loss_arr = []
loss_epoch_arr = []
max_epochs = 10
     
for epoch in range(max_epochs):
     
  for i, data in enumerate(trainloader, 0):
     
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    opt.zero_grad()
     
    outputs = base_model(inputs)
    loss = loss_fn(outputs, labels)
    loss.backward()
    opt.step()
         
    loss_arr.append(loss.item())
        
  loss_epoch_arr.append(loss.item())
         
  print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (epoch, max_epochs, evaluation(testloader), evaluation(trainloader)))

plt.plot(loss_epoch_arr)
plt.show()

In [0]:
torch.save(base_model, '/content/sample_data/my_weights.h5')

In [0]:
from google.colab import files

In [0]:
files.download('/content/sample_data/my_weights.h5')